In [1]:
import keras
from keras import models
from keras import layers
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
!aws s3 sync s3://ulcerclassification .

In [3]:
image_gen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0.0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None)

In [4]:
image_gen = image_gen.flow_from_directory(
    'Data',
    target_size=(300, 300),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=1596,
    shuffle=True,
    seed=123,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    interpolation='nearest')

Found 651 images belonging to 4 classes.


In [5]:
image_gen.class_indices

{'go_to_er': 0, 'healthy_feet': 1, 'needs_TLC': 2, 'see_a_doctor': 3}

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'Data',
        target_size=(150, 150),
        batch_size=32,
        class_mode= 'categorical' )
validation_generator = test_datagen.flow_from_directory(
        'Data',
        target_size=(150, 150),
        batch_size=32,
        class_mode= 'categorical')

Found 651 images belonging to 4 classes.
Found 651 images belonging to 4 classes.


In [7]:
model = Sequential()

In [8]:
model.add(Dense(12, input_dim = 8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='sigmoid'))

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
Total params: 248
Trainable params: 248
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1,
        epochs=4)#,
        #validation_data=validation_generator,
        #validation_steps=800)

train_generator = train_datagen.flow_from_directory(
        'Data',
        target_size=(150, 150),
        batch_size=32,
        class_mode = 'categorical')




Epoch 1/4


ValueError: Error when checking input: expected dense_1_input to have 2 dimensions, but got array with shape (32, 150, 150, 3)

Second Try

Separate data into train and test

In [10]:
# split images and labels
images, labels = next(image_gen)
images.shape, labels.shape

((651, 300, 300, 3), (651, 4))

In [11]:
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size=0.2, random_state=41)
X_model.shape, X_test.shape, y_model.shape, y_test.shape

((520, 300, 300, 3), (131, 300, 300, 3), (520, 4), (131, 4))

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size=0.2, random_state=41)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((416, 300, 300, 3), (104, 300, 300, 3), (416, 4), (104, 4))

In [13]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(64, (3, 3),
                      activation='relu',
                      input_shape=(300, 300,  3),
                      padding='SAME'))
# 64 bias parameters
# 64 * (3 * 3 * 3) weight parametrs
# Output is 64*224*224
cnn.add(layers.MaxPooling2D((2, 2)))
# Output is 64*112*112
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding='SAME'))
# 32 bias parameters
# 32 * (3*3*64)
# Output is 32*112*112
cnn.add(layers.MaxPooling2D((2, 2)))
# Output is 32*56*56
cnn.add(layers.Flatten())
# Output is (100352,)
cnn.add(layers.Dense(32, activation='relu'))
# Output is (32,)
cnn.add(layers.Dense(4, activation='sigmoid'))
# Output is (1,)

cnn.compile(loss='binary_crossentropy',
            optimizer="sgd",
            metrics=['acc'])

In [14]:
saving_weights = keras.callbacks.ModelCheckpoint(
    'weights.{epoch:02d}-{val_loss:.2f}.hdf5',
    monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=True, mode='auto', period=2
)

In [15]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=20,
    verbose=0, mode='auto', min_delta=0.0001, min_lr=0)

In [16]:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=20,
    verbose=0, mode='auto', baseline=None, restore_best_weights=False)

In [17]:
nan_problem = keras.callbacks.TerminateOnNaN()

In [18]:
csv_logger = keras.callbacks.CSVLogger('training.log')

In [19]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 360000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                11520032  
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 132       
Total para

In [ ]:
cnn1 = cnn.fit(X_train,
               y_train,
               epochs=2,
               batch_size=50,
               validation_data=(X_val, y_val),
               callbacks=[csv_logger, early_stop, nan_problem, reduce_lr, saving_weights])




Train on 416 samples, validate on 104 samples
Epoch 1/2







In [1]:
1+1

2

Preprocessing the images

In [57]:
data_te = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "Data",
    target_size=(224, 224),
    batch_size=4419,
    seed=123)

Found 651 images belonging to 4 classes.


In [58]:
data_tr = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'Data',
    target_size=(224, 224),
    batch_size=4419,
    seed=123)

Found 651 images belonging to 4 classes.
